# **Portfolio Optimisation and Backtesting**

## Data wrangling

In this part of the project, we will focus on cleaning the fundamentals data we obtained from Yahoo! Finance.

This step will prepare the data for stock screening, one of the key steps in the project.

## Objectives

In this noebook we will:

*   Handle missing values
*   Format values

<h2>Table of Contents</h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li><a href="#identify_handle_missing_values">Identify and handle missing values</a>
        <ul>
            <li><a href="#identify_missing_values">Identify missing values</a></li>
            <li><a href="#deal_missing_values">Deal with missing values</a></li>
            <li><a href="#correct_data_format">Format currency values</a></li>
        </ul>
    </li>
</ul>

</div>

<hr>

Let's start by importing the necessary libraries.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

Next, we'll import the companies'fundamentals data from `fundamentals.csv`.

In [2]:
df = pd.read_csv('fundamentals.csv')
# Check the first few rows
df.head()

,ticker,name,sector,market_cap,EPS,PE_ratio,ROE,PB_ratio,div_payout_ratio,PS_ratio,FCF,divident_yield,debt_equity,earnings_growth
0,4SI.JO,4Sight Holdings Ltd,Technology,1.581677e+10,-0.015,NaN,0.02316,51.72414,0.0,32.012260,1.271870e+08,NaN,4.158,NaN
1,ABG.JO,Absa Group Limited,Financial Services,1.339503e+13,16.349,987.15515,0.11218,110.10596,0.0,186.482390,NaN,0.0190,NaN,NaN
2,ABSP.JO,ABSA Bank Ltd Pref,Financial Services,3.631238e+11,17.941,4514.79830,0.09397,391.96520,NaN,7.510627,NaN,0.0606,NaN,NaN
3,ACL.JO,ArcelorMittal SA Limited,Basic Materials,1.058880e+12,2.326,408.42648,0.84142,230.07991,0.0,33.911274,1.944500e+09,NaN,148.718,NaN
4,ACS.JO,Acsion Limited,Real Estate,1.647768e+11,1.494,280.45517,0.08583,21.17124,0.0,225.912260,1.832291e+08,NaN,15.684,NaN


Its important that we have an idea of our data before we begin the cleaning process. Let's start with a general info of the dataframe.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ticker            339 non-null    object 
 1   name              336 non-null    object 
 2   sector            328 non-null    object 
 3   market_cap        331 non-null    float64
 4   EPS               323 non-null    float64
 5   PE_ratio          251 non-null    float64
 6   ROE               308 non-null    float64
 7   PB_ratio          314 non-null    float64
 8   div_payout_ratio  294 non-null    float64
 9   PS_ratio          313 non-null    float64
 10  FCF               298 non-null    float64
 11  divident_yield    198 non-null    float64
 12  debt_equity       291 non-null    float64
 13  earnings_growth   51 non-null     float64
dtypes: float64(11), object(3)
memory usage: 38.2+ KB


In [4]:
df.shape

(348, 14)

Already we can see that some of the columns have NaN and zero values which make it hard to analyse our data. 

We intend to screen our stocks using these different attributes, so we'll have to drop columns where there's a lot of missing data (NaNs and zeros). Replacing these values with either the industry median or mean might not be appropriate because these values are the basis of screening similar stocks (in the same sector) when building a diversified portfolio. 

In [5]:
df.columns

Index(['ticker', 'name', 'sector', 'market_cap', 'EPS', 'PE_ratio', 'ROE',
       'PB_ratio', 'div_payout_ratio', 'PS_ratio', 'FCF', 'divident_yield',
       'debt_equity', 'earnings_growth'],
      dtype='object')

We'll capitalise the beginning of each column name for easier referencing and also correct 'divident'. 

In [6]:
df.rename(columns={'ticker':'Ticker', 'name':'Name', 'sector':'Sector', 'market_cap':'Market_cap',
                   'div_payout_ratio':'Div_payout', 'divident_yield':'Div_yield', 'debt_equity':'Debt_to_equity', 
                   'earnings_growth':'Earnings_growth'}, inplace=True)
df.columns

Index(['Ticker', 'Name', 'Sector', 'Market_cap', 'EPS', 'PE_ratio', 'ROE',
       'PB_ratio', 'Div_payout', 'PS_ratio', 'FCF', 'Div_yield',
       'Debt_to_equity', 'Earnings_growth'],
      dtype='object')

## Missing values

We can deal with missing values in three way:
* Look for these values elsewhere - which might not be feasible
* Drop missing values or complete columns if a significant number of values are missing.
* Fill missing values with either median or average values

Let's look at the number of missing values in each column again.

In [7]:
df.isnull().mean().round(2)*100

Ticker              3.0
Name                3.0
Sector              6.0
Market_cap          5.0
EPS                 7.0
PE_ratio           28.0
ROE                11.0
PB_ratio           10.0
Div_payout         16.0
PS_ratio           10.0
FCF                14.0
Div_yield          43.0
Debt_to_equity     16.0
Earnings_growth    85.0
dtype: float64

In [8]:
df.head(10)

,Ticker,Name,Sector,Market_cap,EPS,PE_ratio,ROE,PB_ratio,Div_payout,PS_ratio,FCF,Div_yield,Debt_to_equity,Earnings_growth
0,4SI.JO,4Sight Holdings Ltd,Technology,1.581677e+10,-0.015,NaN,0.02316,51.724140,0.0000,32.012260,1.271870e+08,NaN,4.158,NaN
1,ABG.JO,Absa Group Limited,Financial Services,1.339503e+13,16.349,987.15515,0.11218,110.105960,0.0000,186.482390,NaN,0.0190,NaN,NaN
2,ABSP.JO,ABSA Bank Ltd Pref,Financial Services,3.631238e+11,17.941,4514.79830,0.09397,391.965200,NaN,7.510627,NaN,0.0606,NaN,NaN
3,ACL.JO,ArcelorMittal SA Limited,Basic Materials,1.058880e+12,2.326,408.42648,0.84142,230.079910,0.0000,33.911274,1.944500e+09,NaN,148.718,NaN
4,ACS.JO,Acsion Limited,Real Estate,1.647768e+11,1.494,280.45517,0.08583,21.171240,0.0000,225.912260,1.832291e+08,NaN,15.684,NaN
5,ACT.JO,AfroCentric Inv Corp Ltd,Healthcare,2.917781e+11,0.511,998.04310,0.15491,125.584840,0.6361,36.122425,1.247282e+08,0.0646,30.589,NaN
6,ACZ.JO,Arden Capital Limited,Consumer Cyclical,2.189840e+08,-2.603,NaN,-0.24354,3.220612,0.0000,7.253286,-1.064450e+07,NaN,2.916,NaN
7,ADH.JO,ADvTECH Ltd,Consumer Defensive,9.619250e+11,1.084,1628.22880,0.15461,236.436700,0.1845,173.930920,8.274125e+08,0.0219,46.645,NaN
8,ADI.JO,Adapt IT Holdings Ltd,Industrials,9.676971e+10,0.503,1401.59050,0.08909,120.492226,0.0000,64.368190,2.515491e+08,NaN,65.739,NaN
9,ADR.JO,Adcorp Holdings Limited,Industrials,5.756640e+10,0.275,1949.09080,0.01551,42.753452,0.0000,5.055425,2.624175e+07,NaN,50.370,NaN


We'll drop every row where `Ticker` is NaN.

In [9]:
# Avoid changing original data 
df_edited = df.copy()
# Drop rows with missing values in 'Ticker' and 'Name'.
df_edited.dropna(axis=0, subset=['Ticker', 'Name'], inplace=True)

About 85% of Earnings growth rate numbers are missing from our data. Its difficult to infer values for growth from other variables or sources, so will drop the entire column. Since we have Dividend payout with only 16% of values missing, we can drop Dividend yield which is missing about 43% of values. Although they are different, they're still related and payout ratio seems to be the better indicator when forward looking. 

In [10]:
# Drop columns
df_edited.drop(columns=['Earnings_growth', 'Div_yield'], axis=1, inplace=True)

In Sector, missing values represent REITS, Funds and preference shares mostly. There are less than 10 entries for these. Since we will group values by sector and fill in other missing values based on sector, we will drop missing values from this column.

In [11]:
# Drop rows with missing values in 'Sector'.
df_edited.dropna(axis=0, subset=['Sector'], inplace=True)

Now, lets create a matrix of average values for each metric by sector. We will use these values to replace `NaN` values. Using sector averages for a metric, like EPS, makes more economic sense than using the mean EPS that takes into account all companies since the economic environment is different from one sector to another. It is common practice in investment analysis to compare a company to its peers. 

In [12]:
averages = df_edited.groupby(by='Sector').mean()
averages

,Market_cap,EPS,PE_ratio,ROE,PB_ratio,Div_payout,PS_ratio,FCF,Debt_to_equity
Sector,,,,,,,,,
Basic Materials,1.248492e+13,20.072000,1487.531174,0.068911,1373.530438,0.192100,-23085.992940,4.881616e+09,38.225263
Communication Services,2.706188e+13,58.181286,1916.247662,0.199630,736.721575,0.352279,3787.574550,4.898108e+09,61.189214
Consumer Cyclical,4.801342e+12,1.970225,3200.934361,0.149055,450.652369,0.186592,298.673769,1.466169e+09,169.558154
Consumer Defensive,1.296226e+13,6.827071,6503.456661,0.137500,355.156476,0.802850,435.465481,7.309578e+08,59.400923
Energy,2.583976e+12,7.335800,1486.890904,-0.202518,649.961915,0.089108,36879.912515,1.753740e+10,59.705700
Financial Services,3.700528e+12,7.828774,2640.394759,0.128390,1203.032492,0.297079,NaN,-2.810094e+08,53.877206
Healthcare,2.479475e+12,2.204769,3518.170847,-0.016953,574.613038,0.168850,232.599337,1.069810e+09,195.022667
Industrials,5.634581e+11,5.443191,1211.894971,0.071650,311.344611,0.122700,194.609270,9.097657e+08,106.653533
Real Estate,8.583056e+11,0.432765,1411.273583,0.029610,597.892621,0.397200,6299.832353,1.611233e+08,93.169571


We have a missing `PS_ratio` value for Financial Services. We'll replace it with the median value instead.

In [13]:
median = df_edited[['Sector', 'PS_ratio']].groupby(by='Sector').median().loc['Financial Services', 'PS_ratio']

averages.loc['Financial Services', 'PS_ratio'] = median

Next, we'll iterate over each row of the data and check whether there are any missing values for a particular stock. If a value is missing, we'll replace it with the average value from `averages`. Because its unadvisable to change values were are iterating over, we'll store the return values in lists.

In [14]:
# Create empty lists to store the return values
Ticker = []
Name = []
Sector = []
Market_cap = []
EPS = []
PE_ratio = []
ROE = []
PB_ratio = []
Div_payout = []
PS_ratio = []
FCF = []
Debt_to_equity = []

# For each row (stock) in df_edited
# Ignore index value and get row values only
for _, row in df_edited.iterrows():
    # Return ticker, name and sector as they are
    Ticker.append(row.Ticker)
    Name.append(row.Name)
    Sector.append(row.Sector)
    # Get the stock's sector
    sector = row.Sector
    # Use the sector value and Market_cap to look up average value in averages
    # Add the look up value from averages to return list If the original value is NaN
    # Else (if original value is not NaN), return the original value as it is
    Market_cap.append(averages.loc[sector, 'Market_cap'] if np.isnan(row.Market_cap) else row.Market_cap)
    # Repeat the same for all other metric (column) values
    EPS.append(averages.loc[sector, 'EPS'] if np.isnan(row.EPS) else row.EPS)
    PE_ratio.append(averages.loc[sector, 'PE_ratio'] if np.isnan(row.PE_ratio) else row.PE_ratio)
    ROE.append(averages.loc[sector, 'ROE'] if np.isnan(row.ROE) else row.ROE)
    PB_ratio.append(averages.loc[sector, 'PB_ratio'] if np.isnan(row.PB_ratio) else row.PB_ratio) 
    Div_payout.append(averages.loc[sector, 'Div_payout'] if np.isnan(row.Div_payout) else row.Div_payout)  
    PS_ratio.append(averages.loc[sector, 'PS_ratio'] if np.isnan(row.PS_ratio) else row.PS_ratio) 
    FCF.append(averages.loc[sector, 'FCF'] if np.isnan(row.FCF) else row.FCF) 
    Debt_to_equity.append(averages.loc[sector, 'Debt_to_equity'] if np.isnan(row.Debt_to_equity) else row.Debt_to_equity)

We'll create a new DataFrame `df_filled` using the lists from above.

In [15]:
df_filled = pd.DataFrame(data={'Ticker':Ticker, 'Name':Name, 'Sector':Sector, 'Market_cap':Market_cap, 'EPS':EPS, 'PE_ratio':PE_ratio, 'ROE':ROE, 'PB_ratio':PB_ratio, 
                             'Div_payout':Div_payout, 'PS_ratio':PS_ratio, 'FCF':FCF, 'Debt_to_equity':Debt_to_equity})

Now let's view the outcome of our operation by comparing `df_edited` to `df_filled`.  

In [16]:
df_edited.head()

,Ticker,Name,Sector,Market_cap,EPS,PE_ratio,ROE,PB_ratio,Div_payout,PS_ratio,FCF,Debt_to_equity
0,4SI.JO,4Sight Holdings Ltd,Technology,1.581677e+10,-0.015,NaN,0.02316,51.72414,0.0,32.012260,1.271870e+08,4.158
1,ABG.JO,Absa Group Limited,Financial Services,1.339503e+13,16.349,987.15515,0.11218,110.10596,0.0,186.482390,NaN,NaN
2,ABSP.JO,ABSA Bank Ltd Pref,Financial Services,3.631238e+11,17.941,4514.79830,0.09397,391.96520,NaN,7.510627,NaN,NaN
3,ACL.JO,ArcelorMittal SA Limited,Basic Materials,1.058880e+12,2.326,408.42648,0.84142,230.07991,0.0,33.911274,1.944500e+09,148.718
4,ACS.JO,Acsion Limited,Real Estate,1.647768e+11,1.494,280.45517,0.08583,21.17124,0.0,225.912260,1.832291e+08,15.684


In [17]:
df_filled.head()

,Ticker,Name,Sector,Market_cap,EPS,PE_ratio,ROE,PB_ratio,Div_payout,PS_ratio,FCF,Debt_to_equity
0,4SI.JO,4Sight Holdings Ltd,Technology,1.581677e+10,-0.015,1670.786848,0.02316,51.72414,0.000000,32.012260,1.271870e+08,4.158000
1,ABG.JO,Absa Group Limited,Financial Services,1.339503e+13,16.349,987.155150,0.11218,110.10596,0.000000,186.482390,-2.810094e+08,53.877206
2,ABSP.JO,ABSA Bank Ltd Pref,Financial Services,3.631238e+11,17.941,4514.798300,0.09397,391.96520,0.297079,7.510627,-2.810094e+08,53.877206
3,ACL.JO,ArcelorMittal SA Limited,Basic Materials,1.058880e+12,2.326,408.426480,0.84142,230.07991,0.000000,33.911274,1.944500e+09,148.718000
4,ACS.JO,Acsion Limited,Real Estate,1.647768e+11,1.494,280.455170,0.08583,21.17124,0.000000,225.912260,1.832291e+08,15.684000


Lets check our data for missing values again.

In [18]:
df_filled.isnull().mean().round(2)*100

Ticker            0.0
Name              0.0
Sector            0.0
Market_cap        0.0
EPS               0.0
PE_ratio          0.0
ROE               0.0
PB_ratio          0.0
Div_payout        0.0
PS_ratio          0.0
FCF               0.0
Debt_to_equity    0.0
dtype: float64

In [19]:
df_filled.shape

(328, 12)

## Format values

Currency values for market cap, earnings per share and free cash flow are quoted by Yahoo! Finance in cents. We'll convert Market cap and Free cash flow values to Rand amounts to make it intuitve. We'll leave EPS as it is. 

In [20]:
# Divide all Market_cap values by 100 to convert them to Rands
df_filled.Market_cap = df_filled.Market_cap.apply(lambda x: x/100)
# Do the same for FCF
df_filled.FCF = df_filled.FCF.apply(lambda x: x/100)
# Verify result
df_filled.head()

,Ticker,Name,Sector,Market_cap,EPS,PE_ratio,ROE,PB_ratio,Div_payout,PS_ratio,FCF,Debt_to_equity
0,4SI.JO,4Sight Holdings Ltd,Technology,1.581677e+08,-0.015,1670.786848,0.02316,51.72414,0.000000,32.012260,1.271870e+06,4.158000
1,ABG.JO,Absa Group Limited,Financial Services,1.339503e+11,16.349,987.155150,0.11218,110.10596,0.000000,186.482390,-2.810094e+06,53.877206
2,ABSP.JO,ABSA Bank Ltd Pref,Financial Services,3.631238e+09,17.941,4514.798300,0.09397,391.96520,0.297079,7.510627,-2.810094e+06,53.877206
3,ACL.JO,ArcelorMittal SA Limited,Basic Materials,1.058880e+10,2.326,408.426480,0.84142,230.07991,0.000000,33.911274,1.944500e+07,148.718000
4,ACS.JO,Acsion Limited,Real Estate,1.647768e+09,1.494,280.455170,0.08583,21.17124,0.000000,225.912260,1.832291e+06,15.684000


It worked. Prior, market cap for ABSA Bank was ZAc 363,123,800,000 (3.631238e+11). Now its ZAR 3,631,238,000 (3.631238e+09) - roughly R3.6 billion.

Now that our data is clean, we can start analysing the data and selecting the stocks we want in our portfolio in the next stage of the project. We'll save this data as 'jse_stock_data_cleaned.csv'

In [21]:
df_filled.to_csv('jse_stock_data_cleaned.csv', index=False)